<a href="https://colab.research.google.com/github/alisakar0/car-price-estimation/blob/main/dinamik_fiyatland%C4%B1rma_ve_sat%C4%B1%C5%9F_tahmin_sistemi(lastversion).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
car_path = "/content/drive/MyDrive/Colab Notebooks/car_data3.csv"
df=pd.read_csv(car_path)
df

,Yil,Marka,Model,Yakit,Sanziman,Km,Boya,Degisen,Kaput_Tavan,Fiyat,beygir,Tramer,Hafta
0,2000,FIAT,MAREA,BENZIN,MANUAL,215725,0,0,0,120000,103.00000,4388,9
1,2010,PEUGEOT,206,DIESEL,MANUAL,245682,4,0,0,225000,50.00000,4288,9
2,2012,PEUGEOT,208,BENZIN,MANUAL,75055,3,7,0,216000,60.00000,7341,9
3,2005,FIAT,ALBEA,BENZIN,AUTOMATIC,87590,2,0,0,227000,80.00000,0,9
4,2012,RENAULT,KANGOO,DIESEL,MANUAL,227441,0,1,0,170000,85.00000,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6093,2012,IVECO,35S_13,DIESEL,MANUAL,189144,0,0,0,382000,93.00000,0,26
6094,2005,OPEL,COMBO,DIESEL,MANUAL,186593,4,0,0,180000,78.39804,800,26
6095,2013,RENAULT,CLIO,DIESEL,MANUAL,103950,4,1,0,366000,55.00000,3571,26
6096,2011,SUBARU,FORESTER,BENZIN,AUTOMATIC,58115,2,0,0,697000,110.00000,0,26


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6098 entries, 0 to 6097
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Yil          6098 non-null   int64  
 1   Marka        6098 non-null   object 
 2   Model        6098 non-null   object 
 3   Yakit        6098 non-null   object 
 4   Sanziman     6098 non-null   object 
 5   Km           6098 non-null   int64  
 6   Boya         6098 non-null   int64  
 7   Degisen      6098 non-null   int64  
 8   Kaput_Tavan  6098 non-null   int64  
 9   Fiyat        6098 non-null   int64  
 10  beygir       6098 non-null   float64
 11  Tramer       6098 non-null   int64  
 12  Hafta        6098 non-null   int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 619.5+ KB


In [3]:
missing_values = df.isnull().sum()
print(missing_values)

Yil            0
Marka          0
Model          0
Yakit          0
Sanziman       0
Km             0
Boya           0
Degisen        0
Kaput_Tavan    0
Fiyat          0
beygir         0
Tramer         0
Hafta          0
dtype: int64


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['Arac_Yasi'] = 2024 - X['Yil']
        X['Yiprantis'] = X['Boya'] + X['Degisen']
        return X

In [5]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categorical_cols = ['Marka', 'Model', 'Yakit', 'Sanziman', 'Kaput_Tavan']
numeric_cols = ['Yil', 'Km', 'beygir', 'Tramer', 'Hafta', 'Arac_Yasi', 'Yiprantis']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', StandardScaler(), numeric_cols)
    ],
    remainder='passthrough'
)

In [6]:
from sklearn.model_selection import train_test_split

X = df[['Yil', 'Marka', 'Model', 'Yakit', 'Sanziman', 'Km', 'Boya', 'Degisen', 'Kaput_Tavan', 'beygir', 'Tramer', 'Hafta']]
y = df['Fiyat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression # Import LinearRegression
from sklearn.metrics import mean_squared_error

model_pipeline = Pipeline(steps=[
    ('feature_engineer', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_pipeline.fit(X_train, y_train)

y_pred = model_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Linear Regression MSE: {mse}')

Linear Regression MSE: 25702897706.315582


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = Pipeline(steps=[
    ('feature_engineer', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

model_pipeline.fit(X_train, y_train)

y_pred = model_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Random Forest MSE: {mse}')

Random Forest MSE: 26087845536.179718


In [15]:
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

model = Pipeline(steps=[
    ('feature_engineer', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(tree_method='hist', device='cuda'))
])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'XGBoost MSE: {mse}')


XGBoost MSE: 12703389202.922594


In [16]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline


model_pipeline = Pipeline(steps=[
    ('feature_engineer', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(tree_method='hist', device='cuda'))
])


param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [10, 20, 30],
    'regressor__min_child_weight': [1, 2, 4],
    'regressor__gamma': [0, 0.1, 0.3]
}


grid_search = GridSearchCV(model_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_train, y_train)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score (MSE): {-grid_search.best_score_}')


Best parameters: {'regressor__gamma': 0, 'regressor__max_depth': 10, 'regressor__min_child_weight': 1, 'regressor__n_estimators': 200}
Best score (MSE): 28448737971.002796


In [17]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)


Pipeline(steps=[('feature_engineer', FeatureEngineer()),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Marka', 'Model', 'Yakit',
                                                   'Sanziman', 'Kaput_Tavan']),
                                                 ('num', StandardScaler(),
                                                  ['Yil', 'Km', 'beygir',
                                                   'Tramer', 'Hafta',
                                                   'Arac_Yasi',
                                                   'Yiprantis'])])),
                ('regressor',
                 XGBRegressor(base_sco...
                              feature_types=None, gamma=0, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=10, max_leaves=None, min_child_weight=1,
                              missing=nan, monotone_constraints=None,
                              multi_strategy=None, n_estimators=200,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [18]:
y_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 13865261907.290945
R^2 Score: 0.9399943070461805


In [19]:
import joblib

joblib.dump(best_model, 'best_model.pkl')


['best_model.pkl']

In [21]:
yeni_arac_verileri = pd.DataFrame({
    'Yil': [2012],
    'Marka': ['FIAT'],
    'Model': ['LINEA'],
    'Yakit': ['DIESEL'],
    'Sanziman': ['MANUAL'],
    'Km': [185814],
    'Boya': [4],
    'Degisen': [2],
    'Kaput_Tavan': [0],
    'beygir': [70.0],
    'Tramer': [0],
    'Hafta': [15]
})

fiyat_tahmini = best_model.predict(yeni_arac_verileri)
print(f'Tahmin edilen fiyat: {fiyat_tahmini[0]}')


Tahmin edilen fiyat: 234238.6875
